# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,montevideo,-34.8335,-56.1674,16.57,100,100,5.14,UY,1699663741
1,1,ushuaia,-54.8000,-68.3000,5.81,71,100,7.20,AR,1699663624
2,2,nemuro,43.3236,145.5750,4.51,64,45,10.67,JP,1699663806
3,3,bel ombre,-4.6167,55.4167,27.91,78,40,2.06,SC,1699663806
4,4,ribeira grande,38.5167,-28.7000,20.43,96,17,6.71,PT,1699663806


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 28) &
                                (city_data_df['Humidity'] < 60)]

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()
ideal_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,khargone,21.8167,75.6000,24.83,48,79,0.30,IN,1699663810
21,21,disina,11.4814,9.9190,23.06,26,91,1.53,NG,1699663811
24,24,ramadi,33.4226,43.3058,21.73,36,8,3.04,IQ,1699663593
34,34,stanthorpe,-28.6667,151.9500,25.33,39,0,1.11,AU,1699663813
45,45,i-n-salah,27.1935,2.4607,21.94,38,5,5.82,DZ,1699663815


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
19,khargone,IN,21.8167,75.6000,48,
21,disina,NG,11.4814,9.9190,26,
24,ramadi,IQ,33.4226,43.3058,36,
34,stanthorpe,AU,-28.6667,151.9500,39,
45,i-n-salah,DZ,27.1935,2.4607,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

base_url = "https://api.geoapify.com/v2/places"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city = row["City"] + "," + row["Country"]

    # Add the current city to the parameters
    params["text"] = f"{city}"
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["categories"] = "accommodation.hotel"
    params["filter"] = f"circle:{row['Lng']},{row['Lat']},{radius}"
    params["bias"] = f"proximity:{row['Lng']},{row['Lat']}"
    params["limit"] = 1
    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
khargone - nearest hotel: No hotel found
disina - nearest hotel: No hotel found
ramadi - nearest hotel: فندق الانبار الدولي
stanthorpe - nearest hotel: No hotel found
i-n-salah - nearest hotel: باجودة
geraldton - nearest hotel: Ocean Centre Hotel
alaghsas - nearest hotel: Hôtel Zagado
nguigmi - nearest hotel: Guest PAM
lodwar - nearest hotel: Lodwar Modern Lodge
taoudenni - nearest hotel: No hotel found
tidjikja - nearest hotel: No hotel found
blythe - nearest hotel: Quality Inn
aoulef - nearest hotel: دار الضياف
isna - nearest hotel: No hotel found
al kharijah - nearest hotel: Kharga Hotel
ritchie - nearest hotel: No hotel found
jatai - nearest hotel: Hotel Rio Claro
kara - nearest hotel: Hotel Kara
livingstone - nearest hotel: New Fairmount Hotel and Casino
sanctuary point - nearest hotel: Dungowan Accommodation
mehran - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
de aar - nearest hotel: No hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
19,khargone,IN,21.8167,75.6000,48,No hotel found
21,disina,NG,11.4814,9.9190,26,No hotel found
24,ramadi,IQ,33.4226,43.3058,36,فندق الانبار الدولي
34,stanthorpe,AU,-28.6667,151.9500,39,No hotel found
45,i-n-salah,DZ,27.1935,2.4607,38,باجودة
51,geraldton,AU,-28.7667,114.6000,31,Ocean Centre Hotel
87,alaghsas,NE,17.0187,8.0168,17,Hôtel Zagado
95,nguigmi,NE,14.2495,13.1092,25,Guest PAM
101,lodwar,KE,3.1191,35.5973,52,Lodwar Modern Lodge
108,taoudenni,ML,22.6783,-3.9836,17,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ['Hotel Name','Country']
)

# Display the map plot
map_plot_hotel


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)